In [ ]:
from pathlib import Path
from datetime import date
import sys

import pandas as pd

data_dir = Path("../data")

errors = ['Partial', '!! BAD', 'Inaudible', 'Maybe']

In [ ]:
def count_samples():
    classes = {
        "Errors": 0
    }
        
    classes_files = list(data_dir.glob("**/_classes.csv"))

    for f in classes_files:
        data = pd.read_csv(f, header=None)
        for index, row in data.iterrows():
            if row[1] not in classes:
                classes[row[1]] = 0
            
            if pd.notnull(row[2]) and any([x in row[2] for x in errors]):
                classes["Errors"] += 1
                continue
            else:
                classes[row[1]] += 1
            
            if pd.notnull(row[2]) and not any([x in row[2] for x in errors]):
                for c in row[2].split(", "):
                    if c not in classes:
                        classes[c] = 0
                    classes[c] += 1

    return classes

In [ ]:
def print_info():
    counts = count_samples()
    counts_sorted = sorted(counts, key=counts.get, reverse=True)

    info_file = data_dir / "classes.info"
    with open(info_file, "w+", encoding='utf-8') as info_file:
        info_file.write("Este arquivo enumera e define as classes de segmentos de vídeo presentes no dataset:\n\n")
        info_file.write("Idle    - O depoente se encontra parado ou realizando outra ação irrelevante ao propósito do reconhecedor.\n")
        info_file.write("Speak   - O depoente se encontra falando.\n")
        info_file.write("Nod     - O depoente balança a cabeça como se para dizer que sim.\n")
        info_file.write("Shake   - O depoente balança a cabeça como se para dizer que não.\n")
        info_file.write("Examine - O depoente se aproxima da câmera para examinar alguma evidência que lhe é apresentada.\n")
        info_file.write("\n")
        info_file.write("As seguintes classes estão sob análise devido à falta de vídeos representativos da mesma nos dados diarizados até agora.\n")
        info_file.write("\n")
        info_file.write("KindOf - O depoente balança a cabeça como que para dizer que \"mais ou menos\"\n")
        info_file.write("\n")
        info_file.write("Total de Amostras por Classe (Atualizado em {}):\n".format(date.today()))
        for c in counts_sorted:
            if c is not "Errors":
                info_file.write("{}: {}\n".format(c, counts[c]))
        info_file.write("\n")
        info_file.write("Total de Amostras Inutilizáveis: {}".format(counts["Errors"]))

In [ ]:
print_info()